In [1]:
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('acs2017_census_tract_data.csv')
#df.info()
df = df.dropna()
#df.info()
df[['State', 'County']] = df[['State','County']].apply(LabelEncoder().fit_transform)
df.info()
df.head()
#[.5 points] (1) Load the data into memory and save it to a pandas data frame. Do not normalize or one-hot encode 
#any of the features until asked to do so later in the rubric. (2) Remove any observations that having missing data. 
#(3) Encode any string data as integers for now.(Use label encoder) (4) You have the option of keeping the "county" variable or removing
#it. Be sure to discuss why you decided to keep/remove this variable. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72718 entries, 0 to 74000
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TractId           72718 non-null  int64  
 1   State             72718 non-null  int32  
 2   County            72718 non-null  int32  
 3   TotalPop          72718 non-null  int64  
 4   Men               72718 non-null  int64  
 5   Women             72718 non-null  int64  
 6   Hispanic          72718 non-null  float64
 7   White             72718 non-null  float64
 8   Black             72718 non-null  float64
 9   Native            72718 non-null  float64
 10  Asian             72718 non-null  float64
 11  Pacific           72718 non-null  float64
 12  VotingAgeCitizen  72718 non-null  int64  
 13  Income            72718 non-null  float64
 14  IncomeErr         72718 non-null  float64
 15  IncomePerCap      72718 non-null  float64
 16  IncomePerCapErr   72718 non-null  float6

TractId  State  County  TotalPop   Men  Women  Hispanic  White  Black  \
0  1001020100      0      89      1845   899    946       2.4   86.3    5.2   
1  1001020200      0      89      2172  1167   1005       1.1   41.6   54.5   
2  1001020300      0      89      3385  1533   1852       8.0   61.4   26.5   
3  1001020400      0      89      4267  2001   2266       9.6   80.3    7.1   
4  1001020500      0      89      9965  5054   4911       0.9   77.5   16.4   

   Native  ...  Walk  OtherTransp  WorkAtHome  MeanCommute  Employed  \
0     0.0  ...   0.5          0.0         2.1         24.5       881   
1     0.0  ...   0.0          0.5         0.0         22.2       852   
2     0.6  ...   1.0          0.8         1.5         23.1      1482   
3     0.5  ...   1.5          2.9         2.1         25.9      1849   
4     0.0  ...   0.8          0.3         0.7         21.0      4787   

   PrivateWork  PublicWork  SelfEmployed  FamilyWork  Unemployment  
0         74.2        21.2           4.5         0.0           4.6  
1         75.9        15.0           9.0         0.0           3.4  
2         73.3        21.1           4.8         0.7           4.7  
3         75.8        19.7           4.5         0.0           6.1  
4         71.4        24.1           4.5         0.0           2.3  

[5 rows x 37 columns]

We decided to keep the county variable, however, we consent to the fact that the counties represent political subdivision which are subject to jerrymandering. The tractId is the preferred area identifier employed by the census bureau. 

In [2]:
from sklearn.model_selection import train_test_split

y = df['ChildPoverty']
x = df.drop('ChildPoverty', axis=1)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y, test_size=0.2)

In [3]:
y_train.describe()

count    58174.000000
mean        21.182936
std         18.604172
min          0.000000
25%          6.200000
50%         16.300000
75%         31.700000
max        100.000000
Name: ChildPoverty, dtype: float64

In [4]:
y_train = y_train.div(100)
y_train.describe()

count    58174.000000
mean         0.211829
std          0.186042
min          0.000000
25%          0.062000
50%          0.163000
75%          0.317000
max          1.000000
Name: ChildPoverty, dtype: float64

In [5]:
y_train = pd.qcut(y_train,q=4,labels=['Low','Medium','High','Extreme'],precision=0)
y_train.value_counts()

High       14593
Low        14576
Medium     14519
Extreme    14486
Name: ChildPoverty, dtype: int64

Here we chose to use the pandas qcut to descretize the feature class into 4 equal-sized based on the sample quantiles which represent the low, medium, high, and extreme child poverty rate classes. The balancing should only be done on the training set because we want the testing set to represent never before seen data. 

In [8]:
y_train = pd.get_dummies(y_train,prefix='ChildPoverty')
y_train.head()

ChildPoverty_Low  ChildPoverty_Medium  ChildPoverty_High  \
13544                 0                    0                  0   
26608                 0                    0                  1   
15896                 1                    0                  0   
18963                 0                    0                  0   
47330                 0                    1                  0   

       ChildPoverty_Extreme  
13544                     1  
26608                     0  
15896                     0  
18963                     1  
47330                     0

Here, we one-hot encoded the target training set

In [ ]:
#[.5 points] Balance the dataset so that about the same number of instances are within each class. Choose a method for
#balancing the dataset and explain your reasoning for selecting this method. One option is to choose quantization 
#thresholds for the "ChildPoverty" variable that equally divide the data into four classes. Should balancing of the 
#dataset be done for both the training and testing set? Explain.

In [ ]:
#[.5 points] Assume you are equally interested in the classification performance for each class in the dataset. Split 
#the dataset into 80% for training and 20% for testing. There is NO NEED to split the data multiple times for this lab.
#Note: You will need to one hot encode the target, but do not one hot encode the categorical data until instructed to do so in the lab. 

In [ ]:
#You will be using a two layer perceptron from class for the next few parts of the rubric. There are several versions 
#of the two layer perceptron covered in class, with example code. When selecting an example two layer network from class
#be sure that you use: (1) vectorized gradient computation, (2) mini-batching, (3) cross entropy loss, and (4) proper 
#Glorot initialization, at a minimum. There is no need to use momentum or learning rate reduction (assuming you choose
#a sufficiently small learning rate). It is recommended to use sigmoids throughout the network, but not required.

In [ ]:
#[.5 points] Use the example two-layer perceptron network from the class example and quantify performance using accuracy.
#Do not normalize or one-hot encode the data (not yet). Be sure that training converges by graphing the loss function 
#versus the number of epochs. 

In [ ]:
#[.5 points] Now (1) normalize the continuous numeric feature data. Use the example two-layer perceptron network 
#from the class example and quantify performance using accuracy. Be sure that training converges by graphing the loss 
#function versus the number of epochs.  

In [ ]:
#[.5 points] Now(1) normalize the continuous numeric feature data AND (2) one hot encode the categorical data. Use the
#example two-layer perceptron network from the class example and quantify performance using accuracy. Be sure that 
#training converges by graphing the loss function versus the number of epochs. 

In [ ]:
#[1 points] Compare the performance of the three models you just trained. Are there any meaningful differences in 
#performance? Explain, in your own words, why these models have (or do not have) different performances.  
#Use one-hot encoding and normalization on the dataset for the remainder of this lab assignment.

In [ ]:
#[1 points] Add support for a third layer in the multi-layer perceptron. Add support for saving (and plotting after training 
#is completed) the average magnitude of the gradient for each layer, for each epoch (like we did in the flipped module
#for back propagation). For magnitude calculation, you are free to use either the average absolute values or the L1/L2 norm.
#Quantify the performance of the model and graph the magnitudes for each layer versus the number of epochs.

In [ ]:
#[1 points] Repeat the previous step, adding support for a fourth layer.

In [ ]:
#[1 points] Repeat the previous step, adding support for a fifth layer. 

In [ ]:
#[2 points] Implement an adaptive learning technique that was discussed in lecture and use it on the five layer network
#(such as AdaGrad, RMSProps, or AdaDelta). Discuss which adaptive method you chose. Compare the performance of your five
#layer model with and without the adaptive learning strategy. Do not use AdaM for the adaptive learning technique as it 
#is part of the exceptional work.

In [ ]:
#Exceptional Work (1 points total)
#One idea (required for 7000 level students):  Implement adaptive momentum (AdaM) in the 
#five layer neural network and quantify the performance